## Python implementation of MapReduce

The assumption is that the map-reduce functions follow the following:

`map([x1, x2, ...]) -> [(k1, v2), (k2, v2), ...]`

`reduce([v1, v2, ...]) -> y`

where every element is arbitrary (any data structure)

In [1]:
from itertools import groupby
from operator import itemgetter

def mapreduce(data, map_func, reduce_func):

    # 1. Map phase
    mapped_data = map_func(data)
    
    # 2. Shuffle phase
    mapped_data.sort(key=itemgetter(0))  # Sort input data by key
    grouped_data = groupby(mapped_data, key=itemgetter(0))  # Group data by key
    
    # 3. Reduce
    results = [
        (key, reduce_func([item[1] for item in group])) 
        for key, group in grouped_data
    ]

    return results

## 1. Count characters in words

In [2]:
data = ['dasdsagf', 'mike', 'george', 'gertretr123', 'dsadsajortriojtiow']

def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

In [3]:
mapreduce(data, map_func, reduce_func)

[('1', 1),
 ('2', 1),
 ('3', 1),
 ('a', 4),
 ('d', 4),
 ('e', 5),
 ('f', 1),
 ('g', 4),
 ('i', 3),
 ('j', 2),
 ('k', 1),
 ('m', 1),
 ('o', 4),
 ('r', 6),
 ('s', 4),
 ('t', 4),
 ('w', 1)]

## 2. Word Co-occurence

In this scenario, we want to find out the number of times pairs of words co-occur in the same sentence.

In [4]:
data = ["the quick brown fox jumps over the lazy dog", 
        "the quick blue cat sleeps on the lazy chair", 
        "the quick green bird flies under the lazy cloud"]

def map_func(data):
    result = []
    for sentence in data:
        words = sentence.split()
        for i, word_i in enumerate(words):
            for j, word_j in enumerate(words[i+i:]):
                result.append(
                    ((word_i, word_j), 1)
                )
    return result

def reduce_func(data):
    return sum(data)

In [5]:
mapreduce(data, map_func, reduce_func)

[(('bird', 'cloud'), 1),
 (('bird', 'lazy'), 1),
 (('bird', 'the'), 1),
 (('blue', 'chair'), 1),
 (('blue', 'lazy'), 1),
 (('blue', 'on'), 1),
 (('blue', 'sleeps'), 1),
 (('blue', 'the'), 1),
 (('brown', 'dog'), 1),
 (('brown', 'jumps'), 1),
 (('brown', 'lazy'), 1),
 (('brown', 'over'), 1),
 (('brown', 'the'), 1),
 (('cat', 'chair'), 1),
 (('cat', 'lazy'), 1),
 (('cat', 'the'), 1),
 (('flies', 'cloud'), 1),
 (('fox', 'dog'), 1),
 (('fox', 'lazy'), 1),
 (('fox', 'the'), 1),
 (('green', 'cloud'), 1),
 (('green', 'flies'), 1),
 (('green', 'lazy'), 1),
 (('green', 'the'), 1),
 (('green', 'under'), 1),
 (('jumps', 'dog'), 1),
 (('quick', 'bird'), 1),
 (('quick', 'blue'), 1),
 (('quick', 'brown'), 1),
 (('quick', 'cat'), 1),
 (('quick', 'chair'), 1),
 (('quick', 'cloud'), 1),
 (('quick', 'dog'), 1),
 (('quick', 'flies'), 1),
 (('quick', 'fox'), 1),
 (('quick', 'green'), 1),
 (('quick', 'jumps'), 1),
 (('quick', 'lazy'), 3),
 (('quick', 'on'), 1),
 (('quick', 'over'), 1),
 (('quick', 'sleeps'

## 3. Reverse Web-link Graph

This case assumes we have the graph of web pages where each page links to a list of pages. The goal is to create the reverse graph, where for each page we list all pages linking to it.

In [6]:
data = [("page_A", ["page_B", "page_C", "page_D"]),
        ("page_B", ["page_A", "page_E"]),
        ("page_C", ["page_F", "page_A"])]

def map_func(data):
    result = []
    for page, links in data:
        for link in links:
            result.append((link, page))
    return result

def reduce_func(data):
    return list(data)

In [7]:
mapreduce(data, map_func, reduce_func)

[('page_A', ['page_B', 'page_C']),
 ('page_B', ['page_A']),
 ('page_C', ['page_A']),
 ('page_D', ['page_A']),
 ('page_E', ['page_B']),
 ('page_F', ['page_C'])]

## 4. Document Similarity:

In this scenario, we compute the similarity between pairs of documents using the Jaccard similarity, which measures the overlap between sets.

In [8]:
data = [("doc1", ["the", "quick", "brown", "fox"]),
        ("doc2", ["the", "lazy", "dog"]),
        ("doc3", ["the", "quick", "blue", "cat"])]

def map_func(data):
    result = []
    for document, words in data:
        for word1 in words:
            for word2 in words:
                if word1 != word2:
                    result.append(((word1, word2), 1))
    return result

def reduce_func(data):
    return sum(data)

In [9]:
mapreduce(data, map_func, reduce_func)

[(('blue', 'cat'), 1),
 (('blue', 'quick'), 1),
 (('blue', 'the'), 1),
 (('brown', 'fox'), 1),
 (('brown', 'quick'), 1),
 (('brown', 'the'), 1),
 (('cat', 'blue'), 1),
 (('cat', 'quick'), 1),
 (('cat', 'the'), 1),
 (('dog', 'lazy'), 1),
 (('dog', 'the'), 1),
 (('fox', 'brown'), 1),
 (('fox', 'quick'), 1),
 (('fox', 'the'), 1),
 (('lazy', 'dog'), 1),
 (('lazy', 'the'), 1),
 (('quick', 'blue'), 1),
 (('quick', 'brown'), 1),
 (('quick', 'cat'), 1),
 (('quick', 'fox'), 1),
 (('quick', 'the'), 2),
 (('the', 'blue'), 1),
 (('the', 'brown'), 1),
 (('the', 'cat'), 1),
 (('the', 'dog'), 1),
 (('the', 'fox'), 1),
 (('the', 'lazy'), 1),
 (('the', 'quick'), 2)]

## 5. TF-IDF (Term Frequency-Inverse Document Frequency):

In [10]:
data = [
    ("doc1", "the quick brown fox"),
    ("doc2", "the lazy dog"),
    ("doc3", "the quick blue cat")
]

def map_func(data):
    result = []
    for doc_id, text in data:
        words = text.split()
        term_freq = {}
        for word in words:
            term_freq[word] = term_freq.get(word, 0) + 1
        result.extend([(word, (doc_id, freq)) for word, freq in term_freq.items()])
    return result

def reduce_func(data):
    doc_counts = len(data)
    result = []
    for doc_id, freq in data:
        result.append((doc_id, freq * (1 / doc_counts)))
    return result

In [11]:
mapreduce(data, map_func, reduce_func)

[('blue', [('doc3', 1.0)]),
 ('brown', [('doc1', 1.0)]),
 ('cat', [('doc3', 1.0)]),
 ('dog', [('doc2', 1.0)]),
 ('fox', [('doc1', 1.0)]),
 ('lazy', [('doc2', 1.0)]),
 ('quick', [('doc1', 0.5), ('doc3', 0.5)]),
 ('the',
  [('doc1', 0.3333333333333333),
   ('doc2', 0.3333333333333333),
   ('doc3', 0.3333333333333333)])]

## 6. Distributed Sort

In [12]:
data = [ 
    "record_10", "record_2", "record_3", "record_13",
    "record_1", "record_32", "record_14", "record_18", 
    "record_8", "record_4", "record_12", "record_19",
    "record_13", "record_33", "record_5", "record_9",
]

def map_func(data):
    result = []
    
    for record in data:
        _, key = record.split("_")
        result.append((int(key), record))
    
    return result

def reduce_func(data):
    return data[0]

In [13]:
mapreduce(data, map_func, reduce_func)

[(1, 'record_1'),
 (2, 'record_2'),
 (3, 'record_3'),
 (4, 'record_4'),
 (5, 'record_5'),
 (8, 'record_8'),
 (9, 'record_9'),
 (10, 'record_10'),
 (12, 'record_12'),
 (13, 'record_13'),
 (14, 'record_14'),
 (18, 'record_18'),
 (19, 'record_19'),
 (32, 'record_32'),
 (33, 'record_33')]